# Dataset exploration

In [2]:
import pandas as pd
archivo_csv = 'datos_ideam.csv'
datos_csv = pd.read_csv(archivo_csv)

In [3]:
df = pd.DataFrame(datos_csv)
df

,CodigoEstacion,NombreEstacion,Latitud,Longitud,Altitud,Categoria,Entidad,AreaOperativa,Departamento,Municipio,...,FechaSuspension,IdParametro,Etiqueta,DescripcionSerie,Frecuencia,Fecha,Valor,Grado,Calificador,NivelAprobacion
0,26125710,AEROPUERTO MATECANA [26125710],4.812675,-75.739519,1342,Sinóptica Principal,INSTITUTO DE HIDROLOGIA METEOROLOGIA Y ESTUDIO...,Area Operativa 09 - Cauca-Valle-Caldas,Risaralda,Pereira,...,NaN,RAD SOLAR,RSGVAL_AUT_60,Radiación solar global horaria VALIDADA,Horaria,2014-12-08 15:00,493.0,50,NaN,1200
1,26125710,AEROPUERTO MATECANA [26125710],4.812675,-75.739519,1342,Sinóptica Principal,INSTITUTO DE HIDROLOGIA METEOROLOGIA Y ESTUDIO...,Area Operativa 09 - Cauca-Valle-Caldas,Risaralda,Pereira,...,NaN,RAD SOLAR,RSGVAL_AUT_60,Radiación solar global horaria VALIDADA,Horaria,2014-12-08 16:00,85.0,50,NaN,1200
2,26125710,AEROPUERTO MATECANA [26125710],4.812675,-75.739519,1342,Sinóptica Principal,INSTITUTO DE HIDROLOGIA METEOROLOGIA Y ESTUDIO...,Area Operativa 09 - Cauca-Valle-Caldas,Risaralda,Pereira,...,NaN,RAD SOLAR,RSGVAL_AUT_60,Radiación solar global horaria VALIDADA,Horaria,2014-12-08 17:00,150.2,50,NaN,1200
3,26125710,AEROPUERTO MATECANA [26125710],4.812675,-75.739519,1342,Sinóptica Principal,INSTITUTO DE HIDROLOGIA METEOROLOGIA Y ESTUDIO...,Area Operativa 09 - Cauca-Valle-Caldas,Risaralda,Pereira,...,NaN,RAD SOLAR,RSGVAL_AUT_60,Radiación solar global horaria VALIDADA,Horaria,2014-12-08 18:00,20.8,50,NaN,1200
4,26125710,AEROPUERTO MATECANA [26125710],4.812675,-75.739519,1342,Sinóptica Principal,INSTITUTO DE HIDROLOGIA METEOROLOGIA Y ESTUDIO...,Area Operativa 09 - Cauca-Valle-Caldas,Risaralda,Pereira,...,NaN,RAD SOLAR,RSGVAL_AUT_60,Radiación solar global horaria VALIDADA,Horaria,2014-12-08 19:00,0.0,50,NaN,1200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49138,26125710,AEROPUERTO MATECANA [26125710],4.812675,-75.739519,1342,Sinóptica Principal,INSTITUTO DE HIDROLOGIA METEOROLOGIA Y ESTUDIO...,Area Operativa 09 - Cauca-Valle-Caldas,Risaralda,Pereira,...,NaN,RAD SOLAR,RSGVAL_AUT_60,Radiación solar global horaria VALIDADA,Horaria,2022-09-11 05:00,0.0,50,NaN,1200
49139,26125710,AEROPUERTO MATECANA [26125710],4.812675,-75.739519,1342,Sinóptica Principal,INSTITUTO DE HIDROLOGIA METEOROLOGIA Y ESTUDIO...,Area Operativa 09 - Cauca-Valle-Caldas,Risaralda,Pereira,...,NaN,RAD SOLAR,RSGVAL_AUT_60,Radiación solar global horaria VALIDADA,Horaria,2022-09-11 06:00,1.9,50,NaN,1200
49140,26125710,AEROPUERTO MATECANA [26125710],4.812675,-75.739519,1342,Sinóptica Principal,INSTITUTO DE HIDROLOGIA METEOROLOGIA Y ESTUDIO...,Area Operativa 09 - Cauca-Valle-Caldas,Risaralda,Pereira,...,NaN,RAD SOLAR,RSGVAL_AUT_60,Radiación solar global horaria VALIDADA,Horaria,2022-09-11 07:00,39.7,50,NaN,1200
49141,26125710,AEROPUERTO MATECANA [26125710],4.812675,-75.739519,1342,Sinóptica Principal,INSTITUTO DE HIDROLOGIA METEOROLOGIA Y ESTUDIO...,Area Operativa 09 - Cauca-Valle-Caldas,Risaralda,Pereira,...,NaN,RAD SOLAR,RSGVAL_AUT_60,Radiación solar global horaria VALIDADA,Horaria,2022-09-11 08:00,200.2,50,NaN,1200


In [5]:
#Date must be in 
df['Fecha'] = pd.to_datetime(df['Fecha'])
# Extraer componentes temporales
df['Año'] = df['Fecha'].dt.year
df['Mes'] = df['Fecha'].dt.month

# Crear un promedio mensual de radiación solar
df_mensual = df.groupby(['Año', 'Mes'])['Valor'].mean().reset_index()
df_mensual['Fecha'] = pd.to_datetime(df_mensual[['Año', 'Mes']].assign(Día=1))

# Cargar los datos de El Niño/La Niña
# Asegúrate de usar un archivo que contenga el índice ONI con columnas 'Fecha' y 'ONI'
oni = pd.read_csv('oni_index.csv')
oni['Fecha'] = pd.to_datetime(oni['Fecha'])

# Unir los datos de radiación con los índices ONI
df_combined = pd.merge(df_mensual, oni, on='Fecha', how='inner')

# Crear una columna para clasificar los períodos
def clasificar_fenomeno(oni):
    if oni >= 0.5:
        return 'El Niño'
    elif oni <= -0.5:
        return 'La Niña'
    else:
        return 'Neutral'

df_combined['Fenomeno'] = df_combined['ONI'].apply(clasificar_fenomeno)

# Graficar los promedios mensuales con colores según el fenómeno
plt.figure(figsize=(12, 6))
for fenomeno, grupo in df_combined.groupby('Fenomeno'):
    plt.plot(grupo['Fecha'], grupo['Valor'], label=fenomeno)

plt.title('Radiación Solar vs. Fenómenos El Niño/La Niña')
plt.xlabel('Fecha')
plt.ylabel('Radiación Solar (promedio mensual)')
plt.legend()
plt.grid()
plt.show()

ValueError: to assemble mappings requires at least that [year, month, day] be specified: [day,month,year] is missing